In [11]:
import matplotlib.pyplot as plt

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import mysql.connector
from vnstock import *

def get_news_date(date_start, date_end):
    # Establish a connection to the MySQL database
    connection = mysql.connector.connect(
        host='127.0.0.1',
        port=13306,
        user='root',
        password='root',
        database='pyml'
    )

    # Read the table data using pandas
    query = f"""
        SELECT title, content, date FROM crawl_data
        where DATE(date) >= '{date_start}' and DATE(date) <= '{date_end}'
    """
    df = pd.read_sql(query, connection)
    return df

# Lấy data tin tức từ ngày {start} đến ngày {end} và merge với giá stock thành dataframe.
# Return: dataframe with 3 columns: text, close, date
def get_data_tendency(stock, date_start, date_end, export_url=None):
    df = get_news_date(date_start, date_end)
    # Concatenate columns A and B vertically
    df['text'] = df['title'] + df['content']
    df['date_only'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d')

    df_his = stock_historical_data(stock, date_start, date_end, "1D", 'stock')

    df_his['date'] = pd.to_datetime(df_his['time']).dt.strftime('%Y-%m-%d')
    df_his = df_his.sort_values(by='date', ascending=True)
    # Calculate the percentage increase and store it in a new column
    df_his['percentage_increase'] = df_his['close'].pct_change() * 100
    # Drop the first row since it doesn't have a previous date for comparison
    df_his = df_his.dropna()

    dfMerge = pd.merge(df, df_his, left_on=['date_only'], right_on=['date'], how='inner')
    dfSumarize = dfMerge[['text', 'percentage_increase', 'date_y']]

    # Sorting the DataFrame by the 'date_column' in ascending order

    df_sorted = dfSumarize.sort_values(by='date_y', ascending=True)
    if export_url:
        df_sorted.to_csv(export_url, index=True)

    return df_sorted

In [12]:
data = get_data_tendency('TPB', '2022-01-01', '2023-09-29')
print(data[100:110])

/var/folders/g6/37kt02914kx36yzcbbqfyck00000gn/T/ipykernel_66663/2512981761.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


                                                   text  percentage_increase  \
3506  Doanh nghiệp nước ngoài lạc quan về tương lai ...             1.472810   
3505  Mở đường bay vận chuyển hàng hóa giữa Việt Nam...             1.472810   
3501  Đầu năm, 2 dự án bất động sản 'khủng' nhất TP....             2.642352   
3502  Giá xăng dầu hôm nay 23.2.2022: Tiến gần đến m...             2.642352   
3503  Đất Xanh (DXG): Chủ tịch Lương Trí Thìn sắp ch...             2.642352   
3504  Lịch sự kiện và tin vắn chứng khoán ngày 23/2\...             2.642352   
3500  Có 2 cá nhân chi hơn 76 tỷ đồng mua đấu giá, Đ...            -3.190718   
3499  Website công ty vàng SJC quá tải, vàng ‘bung n...            -3.190718   
3498  Chiến sự nóng ở vựa lúa mì châu Âu, cổ phiếu l...             1.722846   
3497  Vàng SJC ‘đắt’ hơn thế giới 25%, nên bán hay k...             1.722846   

          date_y  
3506  2022-02-22  
3505  2022-02-22  
3501  2022-02-23  
3502  2022-02-23  
3503  2022-02-23  
3504 